In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.cuda
import torch.optim as optim
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import time
import re
from PIL import Image
from tqdm.notebook import tqdm
from torch.nn.functional import relu
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
from sklearn.utils import shuffle
from SHG import SHG
from AE import AE
from SHG_AE import SHG_AE
from utils import *

In [2]:
""" PATHS FOR DATA"""

# Testing data paths
IMGS_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/test/image/"
HEATMAPS_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/test/heatmaps/"

# Used saved model
SHG_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/models/Mon_Apr_12_18-11-59_2021/epoch_46.pth"
SHG_AE_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/SHG_AE/models/Wed_May_26_10-42-05_2021/epoch_15.pth"

In [3]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Average rgb
average_rgb = np.loadtxt("./average_rgb.npy")

# Standard SHG model
SHG_model = SHG(num_hourglasses=1).to(device)
SHG_model.load_state_dict(torch.load(SHG_PATH))
SHG_model.eval()

# SHG + AE
SHG_model_temp = SHG(num_hourglasses = 1).to(device)
AE_model_temp = AE().to(device)
SHG_AE_model = SHG_AE(SHG_model_temp, AE_model_temp)
SHG_AE_model.load_state_dict(torch.load(SHG_AE_PATH)) # loads the model
SHG_AE_model.eval()

Device: cuda


SHG_AE(
  (SHG_model): SHG(
    (pre_conv1): Conv2d(3, 256, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (bn_pre): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pre_residual1): Residual(
      (input_bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
      (branch_cov): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (pre_max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (pre_residual2): Residual(
      (input_bn): BatchNorm2d(256, eps=

In [4]:
class dataset(Dataset):
    def __init__(self, X_path, y_path, average_rgb):
        self.X_path = X_path
        self.y_path = y_path
        self.X_data = os.listdir(self.X_path)
        self.average_rgb = average_rgb
        self.norm = transforms.Normalize(mean = self.average_rgb, std = [1, 1, 1])
        self.order = ['0.npy', '1.npy', '10.npy', '11.npy', '12.npy', '13.npy', '14.npy', '15.npy', '16.npy', '2.npy', '3.npy', '4.npy', '5.npy', '6.npy', '7.npy', '8.npy', '9.npy']

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, i):
        ID = self.X_data[i]
        x = Image.open(self.X_path + ID)

        y = []

        for i in range(17):
            y.append(torch.from_numpy(np.load(self.y_path + ID[:-4] + "/" + str(i) + ".npy")))

        x = TF.to_tensor(x)

        if (x.shape[0] == 1): # If the image is gray-scale, cast it to rgb
            x = torch.stack((x[0],) * 3)

        x = self.norm(x) # Subtracts mean rgb

        y = torch.stack(y)
        return x, y, ID

data = dataset(IMGS_PATH, HEATMAPS_PATH, average_rgb)
dataloader = DataLoader(data, batch_size = 1)

In [5]:
""" Testing on all observations """

criterion = nn.MSELoss()

standard_SHG_loss = 0
standard_SHG_acc = 0

SHG_AE_loss = 0
SHG_AE_acc = 0

with torch.no_grad():
    for x, y, ID in tqdm(dataloader, leave = False, desc = "TESTING...", total = len(dataloader)):
        if (torch.sum(y).item() == 0):
            continue

        x = x.to(device, dtype = torch.float)
        y = y.to(device, dtype = torch.float)

        # === Standard SHG ===

        # Predict
        predictions, _ = SHG_model(x)

        # Loss
        loss = criterion(predictions, y)

        # Storing loss
        standard_SHG_loss += loss.item()

        # Storing accuracy
        standard_SHG_acc += PCK(y.cpu(), predictions.cpu())

        # === SHG + AE ===

        # Predict
        predictions = SHG_AE_model(x, add_noise = False)

        # Loss
        loss = criterion(predictions, y)

        # Storing loss
        SHG_AE_loss += loss.item()

        # Storing accuracy
        SHG_AE_acc += PCK(y.cpu(), predictions.cpu())

    standard_SHG_loss /= len(dataloader)
    standard_SHG_acc /= len(dataloader)

    SHG_AE_loss /= len(dataloader)
    SHG_AE_acc /= len(dataloader)

    print("     Standard SHG loss: {}".format(standard_SHG_loss))
    print("     Standard SHG accuracy: {}".format(standard_SHG_acc))
    print("")
    print("     SHG + AE loss: {}".format(SHG_AE_loss))
    print("     SHG + AE accuracy: {}".format(SHG_AE_acc))

     Standard SHG loss: 5.469225524469701e-05
     Standard SHG accuracy: 0.4412087884849762

     SHG + AE loss: 5.2610600070979105e-05
     SHG + AE accuracy: 0.47400848604152107


In [57]:
""" Testing on only joints with v = 2 """

outputs = np.loadtxt("C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/test/outputs.txt", delimiter = ",", dtype = "str")
outputs_ids = outputs[:, 0]
outputs_kps = outputs[:, np.arange(3, 54, 3)].astype("int")

criterion = nn.MSELoss()

standard_SHG_loss = 0
standard_SHG_acc = 0

SHG_AE_ALL_loss = 0
SHG_AE_ALL_acc = 0

SHG_AE_FULL_loss = 0
SHG_AE_FULL_acc = 0

with torch.no_grad():
    for x, y, ID in tqdm(dataloader, leave = False, desc = "TESTING...", total = len(dataloader)):
        if (torch.sum(y).item() == 0):
            continue

        ID = ID[0][:-4]

        # Finding the groundtruth visibility flags of the keypoints
        row_index = np.where(outputs_ids == ID)
        y_kp = outputs_kps[row_index][0]

        if (2 not in y_kp):
            continue

        # Changes the v = 1 keypoints to v = 0, so they will have no effect on PCK
        if (1 in y_kp):
            y[0, y_kp == 1] = torch.zeros_like(y[0, y_kp == 1])

        x = x.to(device, dtype = torch.float)
        y = y.to(device, dtype = torch.float)

        # === Standard SHG ===

        # Predict
        predictions, _ = SHG_model(x)

        # Loss
        loss = criterion(predictions, y)

        # Storing loss
        standard_SHG_loss += loss.item()

        # Storing accuracy
        standard_SHG_acc += PCK(y.cpu(), predictions.cpu())

        # === SHG + AE ===

        # Predict
        predictions = SHG_AE_model(x, add_noise = False)

        # Loss
        loss = criterion(predictions, y)

        # Storing loss
        SHG_AE_loss += loss.item()

        # Storing accuracy
        SHG_AE_acc += PCK(y.cpu(), predictions.cpu())

    standard_SHG_loss /= len(dataloader)
    standard_SHG_acc /= len(dataloader)

    SHG_AE_loss /= len(dataloader)
    SHG_AE_acc /= len(dataloader)

    print("     Standard SHG loss: {}".format(standard_SHG_loss))
    print("     Standard SHG accuracy: {}".format(standard_SHG_acc))
    print("")
    print("     SHG + AE loss: {}".format(SHG_AE_loss))
    print("     SHG + AE accuracy: {}".format(SHG_AE_acc))

     Standard SHG loss: 5.3499296735990384e-05
     Standard SHG accuracy: 0.4693966345712237

     SHG + AE loss: 5.922715152447656e-05
     SHG + AE accuracy: 0.5755668751818636


In [7]:
""" Testing on all observations, excluding joints related to torso or head """

criterion = nn.MSELoss()

standard_SHG_loss = 0
standard_SHG_acc = 0

SHG_AE_loss = 0
SHG_AE_acc = 0

with torch.no_grad():
    for x, y, ID in tqdm(dataloader, leave = False, desc = "TESTING...", total = len(dataloader)):
        # Letting v = 0 for joints related to torso or head
        y[0, [0, 1, 2, 3, 4, 5, 6, 11, 12]] = torch.zeros_like(y[0, [0, 1, 2, 3, 4, 5, 6, 11, 12]])

        if (torch.sum(y).item() == 0):
            continue

        x = x.to(device, dtype = torch.float)
        y = y.to(device, dtype = torch.float)

        # === Standard SHG ===

        # Predict
        predictions, _ = SHG_model(x)

        # Loss
        loss = criterion(predictions, y)

        # Storing loss
        standard_SHG_loss += loss.item()

        # Storing accuracy
        standard_SHG_acc += PCK(y.cpu(), predictions.cpu())

        # === SHG + AE ===

        # Predict
        predictions = SHG_AE_model(x, add_noise = False)

        # Loss
        loss = criterion(predictions, y)

        # Storing loss
        SHG_AE_loss += loss.item()

        # Storing accuracy
        SHG_AE_acc += PCK(y.cpu(), predictions.cpu())

    standard_SHG_loss /= len(dataloader)
    standard_SHG_acc /= len(dataloader)

    SHG_AE_loss /= len(dataloader)
    SHG_AE_acc /= len(dataloader)

    print("     Standard SHG loss: {}".format(standard_SHG_loss))
    print("     Standard SHG accuracy: {}".format(standard_SHG_acc))
    print("")
    print("     SHG + AE loss: {}".format(SHG_AE_loss))
    print("     SHG + AE accuracy: {}".format(SHG_AE_acc))

     Standard SHG loss: 3.4481039627547795e-05
     Standard SHG accuracy: 0.2531793048973141

     SHG + AE loss: 3.264618633918356e-05
     SHG + AE accuracy: 0.30045371624163064
